# CUSTOMER DATA INPUT VALIDATION AND CLEANING

In [4]:
import pandas as pd
import paramiko, os
from io import StringIO

def read_csv_from_remote_server(host, port, username, password, remote_path):
    try:
        # Create SSH client
        ssh_client = paramiko.SSHClient()

        # Automatically add the server's host key (this is insecure in production)
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the remote server
        ssh_client.connect(hostname=host, port=port, username=username, password=password)

        # Create an SFTP client within the SSH session
        sftp_client = ssh_client.open_sftp()

        # Open the remote CSV file as a file-like object
        with sftp_client.open(remote_path, 'r') as remote_file:
            # Read the CSV content from the remote file
            csv_content = remote_file.read()

        # Create a pandas DataFrame from the CSV content
        df = pd.read_csv(StringIO(csv_content.decode()))

        # Close the SFTP connection
        sftp_client.close()

        return df

    except paramiko.AuthenticationException:
        print("Authentication failed. Please check your username and password.")
    except paramiko.SSHException as ssh_ex:
        print(f"SSH error: {ssh_ex}")
    except Exception as ex:
        print(f"Error: {ex}")

# Replace these values with your WinSCP credentials and the remote CSV file path
host = '172.16.250.72'
port = 22  # Default SSH port is 22
username = 't24tafj'
password = 't24tafj'
remote_path = '/root/T24/APP/bnk/UD/FAFA/DR3 STATIC/R14 _DR3.CUST.INDV.20230721.txt'

# Read the CSV file from the remote server into a pandas DataFrame
customer = read_csv_from_remote_server(host, port, username, password, remote_path)

# Now you can process the DataFrame as needed
#customer.shape


Error: [Errno 13] Permission denied


C:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [1]:
import pandas as pd
from datetime import datetime
import re, os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [48]:
import pandas as pd
import numpy as np
def customer():
    ## Read dataset
    df_cust = pd.read_csv(r"C:\Users\KATAKOF\Downloads\GOLIVE_DATA\R14\STATIC\R14 _GOLIVE.CUST.INDV.20230816.txt",sep='|',encoding='latin-1',dtype="object")
    
    ## Replace all blanks in STREET field with NA
    df_cust["STREET"] = df_cust["STREET"].fillna("NA")
    
    ## Take out duplicates in the NAME_1 field
    df_cust['NAME_1'] = df_cust['NAME_1'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))
    df_cust['SHORT_NAME'] = df_cust['SHORT_NAME'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))
    
    ## Limit number character in "NAME_1" to 35 characters
    df_cust["SHORT_NAME"] = df_cust["SHORT_NAME"].str[:35]
    
    ## Change all dates less than system date in "MANUAL_NEXT_SUIT_REVIEW_DATE" field to system date
    df_cust["MANUAL_NEXT_SUIT_REVIEW_DATE"] = df_cust["MANUAL_NEXT_SUIT_REVIEW_DATE"].replace("20230630" , "20230803")
    
    ## Remove non numeric entry in the CU_EFF_DATE field
    df_cust["CU_EFF_DATE"] = df_cust["CU_EFF_DATE"].replace("20750.0E" , np.nan)
    
    ## Clean up all unwanted characters in the field "AUTH_POL_EXPO" 
    df_cust['AUTH_POL_EXPO'] = df_cust['AUTH_POL_EXPO'].str.replace(r'.*!', '', regex=True)
    df_cust["AUTH_POL_EXPO"] = df_cust["AUTH_POL_EXPO"].replace({"MEDDIUM HIGH RISK":"MEDIUM HIGH RISK"})
    
    ## Take out all customer_number instructed by Retail as duplicate customers
#     df_dup = pd.read_csv(r"C:\Users\KATAKOF\Downloads\GOLIVE_DATA\R14\STATIC\IDstobeRemoved.csv",dtype="object")
#     df_cust = df_cust[~df_cust["@ID"].isin(list(df_dup["IDs"]))]
    
    
#     df_nost = pd.read_csv(r"R14 _GOLIVE.CUST.NOST.20230816\R14 _GOLIVE.CUST.NOST.20230816.txt",sep='|',encoding='latin-1',dtype="object")  
#     df_cust_out = df_nost[~df_nost["DATE_OF_BIRTH"].isnull()]
    
#     df_cust = df_cust.append(df_cust_out)
    
        # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'customer.txt')
    df_cust.to_csv(file_path,sep='|', index=False)
    
    return df_cust

In [49]:
df_cust = customer()

In [51]:
df_cust.shape

(470280, 307)

In [52]:
# df_cust = pd.read_csv(r"C:\Users\KATAKOF\Downloads\GOLIVE_DATA\R14\STATIC\R14 _GOLIVE.CUST.INDV.20230816.txt",sep='|',encoding='latin-1',dtype="object")
df_dup = pd.read_csv(r"C:\Users\KATAKOF\Downloads\GOLIVE_DATA\R14\STATIC\IDstobeRemoved.csv",dtype="object")

In [43]:
df_dup.shape

(242, 1)

In [53]:
df_final = df_cust[~df_cust["@ID"].isin(list(df_dup["IDs"]))]

In [54]:
print(df_cust.shape)
print(df_dup.shape)
print(df_final.shape)

(470280, 307)
(242, 1)
(470044, 307)


In [57]:
df_nost = pd.read_csv(r"C:\Users\KATAKOF\Downloads\GOLIVE_DATA\R14\STATIC\R14 _GOLIVE.CUST.NOST.20230816.txt",sep='|',encoding='latin-1',dtype="object")  
df_cust_out = df_nost[~df_nost["DATE_OF_BIRTH"].isnull()]

In [60]:
df_final_ = df_final.append(df_cust_out)

In [61]:
df_final_.shape

(470134, 307)

In [62]:
folder_name = 'cleaned_data'
if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
file_path = os.path.join(folder_name, 'customer.txt')
df_final_.to_csv(file_path,sep='|', index=False)

## NOSTRO ACCOUNTS

In [4]:
df_nost = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\GOLIVE DATA\ORIGINAL FILES\STATIC\R14 _GOLIVE.CUST.NOST.20230801.txt",sep='|',encoding='latin-1',dtype="object")
df_nostro = df_nost[df_nost["DATE_OF_BIRTH"].isnull()]
folder_name = 'cleaned_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

 
##
##Export DataFrame data to a CSV file inside the folder
file_path = os.path.join(folder_name, 'nostro_customer.txt')
df_cust.to_csv(file_path,sep='|', index=False)

In [31]:
df_cust_out.shape

(88, 307)

In [19]:
df_nost.shape

(492, 307)

In [ ]:
# Create a folder (if it doesn't exist)
folder_name = 'cleaned_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Export DataFrame data to a CSV file inside the folder
file_path = os.path.join(folder_name, 'cleaned_mm_data.txt')
mm_data.to_csv(file_path,sep='|', index=False)

In [125]:
df_cust_1["AUTH_POL_EXPO"].value_counts()

LOW RISK            325519
MEDIUM RISK          58953
MEDIUM HIGH RISK     14919
HIGH RISK            10214
Name: AUTH_POL_EXPO, dtype: int64

In [101]:
col = []

for co in df_cust:
    col.append(co)

In [103]:
col.sort()

In [104]:
col

['@ID',
 'ACCOUNT_OFFICER',
 'ADDRESS',
 'ALLOW_BULK_PROCESS',
 'AMLCUS_DOB',
 'AMLCUS_GENDER',
 'AMLCUS_ID_AUTH',
 'AMLCUS_ID_CO',
 'AMLCUS_ID_EDATE',
 'AMLCUS_ID_NO',
 'AMLCUS_ID_SDATE',
 'AMLCUS_ID_TYPE',
 'AMLCUS_NAME',
 'AMLCUS_NATION',
 'AMLCUS_OCC',
 'AMLCUS_PHONE_NO',
 'AMLCUS_RES_ADD',
 'AMLCUS_ROLE',
 'AML_CHECK',
 'AML_RESULT',
 'ANNUAL_BONUS',
 'ANNUAL_TURNOVER',
 'APPL_POL_EXP',
 'APPL_VERSION',
 'ATM',
 'AUTH_POL_EXPO',
 'AUTO_NEXT_KYC_REVIEW_DATE',
 'AUTO_NEXT_SUIT_REVIEW_DATE',
 'BIRTH_INCORP_DATE',
 'BUSINESS_ENTITY',
 'BUS_COM_DATE',
 'BUS_COM_NUM',
 'BUS_REG_DATE',
 'CHANGE_DATE',
 'CHANGE_REASON',
 'COMPANY_BOOK',
 'COMP_AC_TAG1',
 'COMP_REG_NO',
 'CONTACT_DATE',
 'CONTACT_NAME',
 'CONT_NAME',
 'CONT_PER',
 'CONT_PERSON',
 'CONT_TEL',
 'CONT_TELE',
 'CORR_NAME',
 'COUNTRY',
 'CP_DATEOFBIRTH',
 'CP_EMAIL_ADD',
 'CP_FIRST_NAME',
 'CP_GENDER',
 'CP_IDNO',
 'CP_JOB_TITLE',
 'CP_MOB_NO',
 'CP_NATIONALITY',
 'CP_OCCUPATION',
 'CP_POST_ADDR',
 'CP_RESID_ADD',
 'CP_SURNAME'

In [118]:
df_cust_1["MANUAL_NEXT_SUIT_REVIEW_DATE"].unique().max()

'20230630'

In [67]:
# df_custa.STREET.unique

In [ ]:
20230612

In [7]:
def mm_number(row):
    currency = row['Currency']
    code = row['Code']
    return currency + code[3:]
#    data['Code'] = data.apply(mm_number, axis=1)
# Custom function to replace values
'''def replace_with_today(date_str):
    #date_integer = int(datetime.strptime(date_str, '%Y-%m-%d').strftime('%Y%m%d'))
    date_integer = int(date_str)
    return today_integer if today_integer < date_integer else date_integer'''

def replace_with_today(date_str):
    today = datetime.today()
    today_integer = int(today.strftime('%Y%m%d'))

    if pd.isnull(date_str):
        date_integer = 0
    else:
        try:
            date_integer = int(date_str)
        except (ValueError, TypeError):
            try:
                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                date_integer = int(date_obj.strftime('%Y%m%d'))
            except (ValueError, TypeError):
                date_integer = 0
    system_date= 20230721
    return today_integer if today_integer < date_integer else date_integer


def customer_data():
    data = pd.read_csv("R14 _DR3.CUST.INDV.20230721.txt",sep='|',encoding='latin-1',dtype="object")
    nostros = pd.read_csv("R14 _DR3.CUST.NOST.20230721.txt",sep='|',dtype='object')
    update_customer = nostros[~nostros["DATE_OF_BIRTH"].isna()]
    update_nostros = nostros[nostros["DATE_OF_BIRTH"].isna()]
    # Append data to update_customer
    data = pd.concat([data, update_customer], ignore_index=True)
    data['STREET'] = data['STREET'].fillna("NA")
    duplicate = pd.read_excel("Duplicate Customers.xlsx",dtype='object')
    condition = data["@ID"].isin(duplicate['DUPLICATES'])
    data = data[~condition]
    # Remove text before and including the last '!'
    data['AUTH_POL_EXPO'] = data['AUTH_POL_EXPO'].str.replace(r'.*!', '', regex=True)
    data["AUTH_POL_EXPO"] = data["AUTH_POL_EXPO"].replace({"MEDDIUM HIGH RISK":"MEDIUM HIGH RISK"})
    data["LEGAL_HOLDER_NAME"] = data["LEGAL_HOLDER_NAME"].str.slice(0, 100)
    data["FIRST_NAME"] = data["FIRST_NAME"].str.slice(0, 100)
    data["MIDDLE_NAME"] = data["MIDDLE_NAME"].str.slice(0, 100)
    data["NAME_1"] = data["NAME_1"].str.slice(0, 100)
    data["NAME_2"] = data["NAME_2"].str.slice(0, 100)
    ### Contact date validation
    # Get today's date
    today = datetime.today().date()
    #Convert the date to an integer
    today_integer = int(today.strftime('%Y%m%d'))
    # Apply the function to the "date" column
    data['CONTACT_DATE'] = data['CONTACT_DATE'].apply(replace_with_today)
    data['LAST_KYC_REVIEW_DATE'] = data['LAST_KYC_REVIEW_DATE'].apply(replace_with_today)
    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'cleaned_customer_data.txt')
    data.to_csv(file_path,sep='|', index=False)
    update_nostros.to_csv("cleaned_data/updated_nostros.txt",sep='|',index=False)
    return data

In [8]:
customer_data = customer_data()
customer_data.shape

(468795, 307)

In [70]:
#condition = customer_data["@ID"].isin(duplicate['DUPLICATES'])
#customer_data1 = customer_data[~condition]
#customer_data1.shape

(468707, 307)

In [55]:
data.AUTH_POL_EXPO.unique()

array(['LOW RISK', nan, 'MEDIUM HIGH RISK', 'MEDIUM RISK', 'HIGH RISK'],
      dtype=object)

In [66]:
customer_data[customer_data['@ID']==552525]

,UPLOAD_COMPANY,@ID,MNEMONIC,SHORT_NAME,NAME_1,NAME_2,STREET,TOWN_COUNTRY,SECTOR,ACCOUNT_OFFICER,OTHER_OFFICER,INDUSTRY,TARGET,NATIONALITY,CUSTOMER_STATUS,RESIDENCE,CONTACT_DATE,INTRODUCER,TEXT,LEGAL_ID,REVIEW_FREQUENCY,BIRTH_INCORP_DATE,POSTING_RESTRICT,POST_CODE,COUNTRY,COMPANY_BOOK,CUSTOMER_RATING,ADDRESS,TITLE,GIVEN_NAMES,FAMILY_NAME,GENDER,DATE_OF_BIRTH,MARITAL_STATUS,NO_OF_DEPENDENTS,PHONE_1,SMS_1,EMAIL_1,EMPLOYMENT_STATUS,OCCUPATION,EMPLOYERS_NAME,EMPLOYERS_ADD,EMPLOYERS_BUSS,EMPLOYMENT_START,CUSTOMER_CURRENCY,SALARY,ANNUAL_BONUS,SALARY_DATE_FREQ,NET_MONTHLY_IN,NET_MONTHLY_OUT,RESIDENCE_STATUS,RESIDENCE_TYPE,RESIDENCE_SINCE,RESIDENCE_VALUE,FAX_1,PREVIOUS_NAME,CHANGE_DATE,CHANGE_REASON,CUSTOMER_SINCE,CUSTOMER_TYPE,DATE_LAST_VERIFIED,REL_DELIV_OPT,ROLE,ROLE_MORE_INFO,ROLE_NOTES,LEGAL_DOC_NAME,LEGAL_ISS_AUTH,LEGAL_ISS_DATE,LEGAL_EXP_DATE,OFF_PHONE,LEGAL_HOLDER_NAME,JOB_TITLE,PASTIMES,FURTHER_DETAILS,DOMICILE,OTHER_NATIONALITY,TAX_ID,RISK_LEVEL,RISK_TOLERANCE,LAST_KYC_REVIEW_DATE,AUTO_NEXT_KYC_REVIEW_DATE,MANUAL_NEXT_SUIT_REVIEW_DATE,LAST_SUIT_REVIEW_DATE,AUTO_NEXT_SUIT_REVIEW_DATE,KYC_RELATIONSHIP,MANDATE_APPL,MANDATE_REG,MANDATE_RECORD,SECURE_MESSAGE,AML_CHECK,AML_RESULT,LAST_AML_RESULT_DATE,KYC_COMPLETE,INTERNET_BANKING_SERVICE,MOBILE_BANKING_SERVICE,ALLOW_BULK_PROCESS,MERGED_TO,MERGED_STATUS,PERSON_ENTITY_ID,SEGMENT,CU_EFF_DATE,MAIDEN_NAME,MAJ_LAND_MARK,WEB_ADDRESS,FIRST_NAME,EMPL_FON,EMPL_EMAIL,SOCIAL_SEC_NO,SURNAME_NON_CUS,FIRST_NAME_NC,MIDDLE_NAME,DTE_OF_BIRTH_NC,GENDER_NC,RELATIONSHIP_NC,RESIDENT_ADD_NC,POSTAL_ADD_NC,MOBILE_NO_NC,ATM,PLACE_OF_INCORP,PARENT_COMP,PAR_COMP_INCORP,WEBSITE,VAT_REG_NO,CP_SURNAME,CP_FIRST_NAME,CP_DATEOFBIRTH,CP_GENDER,CP_OCCUPATION,CP_JOB_TITLE,CP_NATIONALITY,CP_RESID_ADD,CP_POST_ADDR,CP_MOB_NO,CP_EMAIL_ADD,PROOF_OF_ADD,ANNUAL_TURNOVER,SOURCE_OF_FUNDS,GSE_LISTING,GSE_NO,D_SURNAME,D_FIRST_NAME,D_GENDER,D_DATE_OF_BIRTH,D_OCCUPATION,D_NATIONALITY,D_RESI_PERMIT,D_ID_TYPE,D_ID_NUMBER,D_RESID_ADD,D_POST_ADDR,D_MOB_NO,D_EMAIL,D_DIR_STATUS,S_ENTITY_NAME,S_CONT_INCOR,S_BUS_ADDR,S_POST_ADDR,S_TEL,S_STATUS,S_BENEFI,S_REGI_CERT,S_EMAIL,C_TRANS_ALERT,C_STATE_PREF,C_STATE_FREQ,O_NAME_OF_BANK,O_BRANCH,O_ACCT_NAME,OTHER_ACCT,OTHER_ACCT_NO,NXT_KIN_SURNAME,NXT_KIN_FNAME,NXT_KIN_MNAME,NXT_KIN_DOB,NXT_KIN_GENDER,NXT_KIN_FON,NXT_KIN_MOB,COMP_REG_NO,PLACE_OF_BIRTH,REGION,CUS_PUR_ACCT,NUM_CURR,CUS_CHQ_REQ,CUS_CHQ_CONF,CUS_TRANS_ALERT,CUS_STMT_FRQ,CUS_CHQ_THRES,MOBILE_NUMBER,CUS_ACCT_BRANCH,CUS_ACCT_NAME,CUS_ACCT_NO,CUS_ACCT_TYPE,OTH_BNK_NAME,OTH_BNK_BRCH,OTH_ACC_NAME,OTH_ACC_NUM,OTH_ACC_TYPE,CUS_BUS_REG_NO,RESI_PER_NO,RESI_PER_DATE,CUS_PLA_ISSUE,CUS_TAX_ID,CUS_SSNIT_NO,CUS_M_MAIDEN_NA,CUS_PREV_NAME,CP_IDNO,APPL_POL_EXP,AUTH_POL_EXPO,CUS_DIR,IF_DIR_CUST,IND_SH_SN,IND_SH_FN,IND_SH_PA,IND_SH_PN,IND_SH_ST,CUS_ACCT_OTH_BK,APPL_VERSION,HSE_NO,CONT_PER,CONT_TEL,BUS_COM_DATE,BUS_COM_NUM,BUS_REG_DATE,CONTACT_NAME,CONT_PERSON,CONT_TELE,CONT_NAME,EZWICH_NO,RESIDENTIAL_ADD,POSTAL_ADD,CORR_NAME,ENTITY_TYPE,CUST_DUMMY,FOR_STATE,FOR_CITY,FOR_PHONE_HOUSE,FOR_MOB_NO,FOR_ADD,FOR_MAIL_ADD,FOR_TIN,FOR_SSN,FOR_GREEN_CARD,FOR_POA,FOR_INST,FOR_ZIP,FOR_CITIZEN,FOR_PERM_RESD,FOR_BIRTH_PLACE,FOR_COR_M_ADD,FOR_POA_SIGN,FOR_BORN,FOR_INST_FT,FOR_SUB_PRES,FOR_ENTITY,FOR_ENT_TYPE,FOR_ENT_OWNER,PO_REASON,TIN_NO,AMLCUS_GENDER,AMLCUS_NAME,AMLCUS_DOB,AMLCUS_PHONE_NO,AMLCUS_NATION,AMLCUS_RES_ADD,AMLCUS_OCC,AMLCUS_ROLE,AMLCUS_ID_TYPE,AMLCUS_ID_NO,AMLCUS_ID_CO,AMLCUS_ID_AUTH,AMLCUS_ID_SDATE,AMLCUS_ID_EDATE,CUST_SHAREHOLD,SHARE_NATIONAL,CUST_DIRECTOR,CUST_DIR_NA,CUST_SIGNATOR,CUST_SIG_NA,CUST_SIG_KCP,CUST_UBO,CUST_UBO_NA,CUST_EVT_INFO,CUST_ANNIVERSAR,NATURE_OF_BUSIN,NATURE_BUS_CERT,NATURE_EXPIRY,COMP_AC_TAG1,BUSINESS_ENTITY,RESE_PER_DATE,KYC_DEF_STATUS,KYC_DEF_REG_STA,CUS_CHAL_STATUS,CUS_REL_STATUS,CUS_REL_BANK,CUS_RES_STATUS,CUS_DIGI_ADD,DEP_CLASS_TYPE,PCUS_CARD_PURP,PCUS_DEP_FREQ,PCUS_DISABILITY,PCUS_EXP_LOAD,PCUS_F_SOURCE,PCUS_P_OF_BIRTH,PCUS_PRF_SOURCE,DETAILS_CHECK,REFER_INFLOW,GHA_CARD
422889,GH0010001,552525,UM00552525,ADJEI EMMANUEL ANTWI,ADJ

# MONEY MARKET - MM VALIDATION

In [76]:
def mm_number(row):
    currency = row['CURRENCY']
    code = row['DRAWDOWN_ACCOUNT']
    return currency + code[3:]

mm_data = pd.read_csv("R14 _DR2.MM.20230721.txt",sep='|',encoding='latin-1',)
mm_data['DRAWDOWN_ACCOUNT'] = mm_data.apply(mm_number, axis=1)

# Create a folder (if it doesn't exist)
folder_name = 'cleaned_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Export DataFrame data to a CSV file inside the folder
file_path = os.path.join(folder_name, 'cleaned_mm_data.txt')
mm_data.to_csv(file_path,sep='|', index=False)

In [74]:
mm_data.head()

,UPLOAD_COMPANY,@ID,CUSTOMER_ID,ROLLOVER_MARKER,CURRENCY,PRINCIPAL,DEAL_DATE,VALUE_DATE,MATURITY_DATE,CATEGORY,INT_RATE_TYPE,INTEREST_BASIS,INTEREST_RATE,INTEREST_SPREAD_1,OUR_REMARKS,DRAWDOWN_ACCOUNT,PRIN_LIQ_ACCT,PRIN_BEN_BANK_1,PRIN_BEN_BANK_2,PRIN_ADDRESS,BENEF_PRIN_ACCT,INT_LIQ_ACCT,INT_BEN_BANK_1,INT_BEN_BANK_2,INT_ADDRESS,BEN_INT_ACCT,TAX_INTEREST_KEY,LIMIT_REFERENCE,DELIVERY_LINK,CHRG_LIQ_ACCT,CHARGE_CODE,CHARGE_AMOUNT,CURRENCY_MARKET,COUNTRY_RISK,COUNTRY_EXPOSURE,MIS_ACCT_OFFICER,NEW_INTRST_SPREAD,CAPITALISATION,CONF_BY_CUST,CONF_BY_BROKER,DEALER_DESK,INT_CCY,INT_CCY_MKT,AUTO_ROLLOVER,AUTO_CAPITALISE,AUTO_ROLL_TERM,ROLLOVER_INT_RATE,LIQUIDATION_MODE,NEXT_INT_TAX_AMT,NEXT_PRIN_AMOUNT,ORIG_START_DATE,ORIG_MAT_DATE,PREV_MAT_DATE,MI_FUNDING_DEPT,TAX_INTEREST_TYPE,MATURE_AT_SOD,MATURED_ONLINE,ROUNDING_RULE,SPEC_INT_METHOD,ACCRUAL_PARAM,TENOR_DEPOSIT,APPL_VERSION,MM_CERT_NO,MM_CERT_DATE,MM_TYPE
0,GH0010009,MM1615301007,105829,NaN,GHS,2865.49,20160601,20160601,20231122,21002,1,E,6.5,NaN,NaN,GHS1499800040009,0102105829016,NaN,NaN,NaN,NaN,0102105829016,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,106926,NaN,N,NaN,NaN,0,GHS,1,Y,Y,182D,6.5,AUTOMATIC,NaN,NaN,20160601,20161130,NaN,NaN,*WHTDEP,YES,NO,NATURAL,NaN,FIRST,182.0,",FDAMEND",NaN,NaN,NaN
1,GH0010004,MM2308729168,71451,NaN,GHS,206410.96,20230328,20230328,20230925,21002,1,E,13.0,NaN,NaN,GHS1499800040004,0051071451015,NaN,NaN,NaN,NaN,0051071451015,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108210,NaN,N,NaN,NaN,0,GHS,1,Y,Y,90D,NaN,AUTOMATIC,NaN,NaN,20230328,20230626,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,90.0,",FD",NaN,NaN,NaN
2,GH0010034,MM2313929698,554310,NaN,GHS,50000.00,20230519,20230519,20230818,21002,1,E1,13.0,NaN,NaN,GHS1499800040034,0332554310012,NaN,NaN,NaN,NaN,0332554310012,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,120285,NaN,N,NaN,NaN,0,GHS,1,N,N,NaN,NaN,AUTOMATIC,NaN,NaN,20230519,20230818,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,91.0,",FDX",NaN,NaN,NaN
3,GH0010012,MM2230027397,88802,NaN,GHS,160097.26,20221027,20221027,20231026,21002,1,E,13.5,NaN,NaN,GHS1499800040012,0051088802016,NaN,NaN,NaN,NaN,0051088802016,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108210,NaN,N,NaN,NaN,0,GHS,1,Y,Y,182D,NaN,AUTOMATIC,NaN,NaN,20221027,20230427,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,182.0,",FD",NaN,NaN,NaN
4,GH0010026,MM2320130348,915078,NaN,GHS,300000.00,20230720,20230720,20231018,21002,1,E,13.0,NaN,NaN,GHS1499800040026,0272915078017,NaN,NaN,NaN,NaN,0272915078017,NaN,NaN,NaN,NaN,8.0,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108416,NaN,N,NaN,NaN,0,GHS,1,Y,N,90D,NaN,AUTOMATIC,NaN,NaN,20230720,20231018,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,90.0,",FD",NaN,NaN,NaN


In [77]:
mm_data.head()

,UPLOAD_COMPANY,@ID,CUSTOMER_ID,ROLLOVER_MARKER,CURRENCY,PRINCIPAL,DEAL_DATE,VALUE_DATE,MATURITY_DATE,CATEGORY,INT_RATE_TYPE,INTEREST_BASIS,INTEREST_RATE,INTEREST_SPREAD_1,OUR_REMARKS,DRAWDOWN_ACCOUNT,PRIN_LIQ_ACCT,PRIN_BEN_BANK_1,PRIN_BEN_BANK_2,PRIN_ADDRESS,BENEF_PRIN_ACCT,INT_LIQ_ACCT,INT_BEN_BANK_1,INT_BEN_BANK_2,INT_ADDRESS,BEN_INT_ACCT,TAX_INTEREST_KEY,LIMIT_REFERENCE,DELIVERY_LINK,CHRG_LIQ_ACCT,CHARGE_CODE,CHARGE_AMOUNT,CURRENCY_MARKET,COUNTRY_RISK,COUNTRY_EXPOSURE,MIS_ACCT_OFFICER,NEW_INTRST_SPREAD,CAPITALISATION,CONF_BY_CUST,CONF_BY_BROKER,DEALER_DESK,INT_CCY,INT_CCY_MKT,AUTO_ROLLOVER,AUTO_CAPITALISE,AUTO_ROLL_TERM,ROLLOVER_INT_RATE,LIQUIDATION_MODE,NEXT_INT_TAX_AMT,NEXT_PRIN_AMOUNT,ORIG_START_DATE,ORIG_MAT_DATE,PREV_MAT_DATE,MI_FUNDING_DEPT,TAX_INTEREST_TYPE,MATURE_AT_SOD,MATURED_ONLINE,ROUNDING_RULE,SPEC_INT_METHOD,ACCRUAL_PARAM,TENOR_DEPOSIT,APPL_VERSION,MM_CERT_NO,MM_CERT_DATE,MM_TYPE
0,GH0010009,MM1615301007,105829,NaN,GHS,2865.49,20160601,20160601,20231122,21002,1,E,6.5,NaN,NaN,GHS1499800040009,0102105829016,NaN,NaN,NaN,NaN,0102105829016,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,106926,NaN,N,NaN,NaN,0,GHS,1,Y,Y,182D,6.5,AUTOMATIC,NaN,NaN,20160601,20161130,NaN,NaN,*WHTDEP,YES,NO,NATURAL,NaN,FIRST,182.0,",FDAMEND",NaN,NaN,NaN
1,GH0010004,MM2308729168,71451,NaN,GHS,206410.96,20230328,20230328,20230925,21002,1,E,13.0,NaN,NaN,GHS1499800040004,0051071451015,NaN,NaN,NaN,NaN,0051071451015,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108210,NaN,N,NaN,NaN,0,GHS,1,Y,Y,90D,NaN,AUTOMATIC,NaN,NaN,20230328,20230626,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,90.0,",FD",NaN,NaN,NaN
2,GH0010034,MM2313929698,554310,NaN,GHS,50000.00,20230519,20230519,20230818,21002,1,E1,13.0,NaN,NaN,GHS1499800040034,0332554310012,NaN,NaN,NaN,NaN,0332554310012,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,120285,NaN,N,NaN,NaN,0,GHS,1,N,N,NaN,NaN,AUTOMATIC,NaN,NaN,20230519,20230818,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,91.0,",FDX",NaN,NaN,NaN
3,GH0010012,MM2230027397,88802,NaN,GHS,160097.26,20221027,20221027,20231026,21002,1,E,13.5,NaN,NaN,GHS1499800040012,0051088802016,NaN,NaN,NaN,NaN,0051088802016,NaN,NaN,NaN,NaN,NaN,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108210,NaN,N,NaN,NaN,0,GHS,1,Y,Y,182D,NaN,AUTOMATIC,NaN,NaN,20221027,20230427,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,182.0,",FD",NaN,NaN,NaN
4,GH0010026,MM2320130348,915078,NaN,GHS,300000.00,20230720,20230720,20231018,21002,1,E,13.0,NaN,NaN,GHS1499800040026,0272915078017,NaN,NaN,NaN,NaN,0272915078017,NaN,NaN,NaN,NaN,8.0,9900.01,1,NaN,NaN,NaN,1,NaN,NaN,108416,NaN,N,NaN,NaN,0,GHS,1,Y,N,90D,NaN,AUTOMATIC,NaN,NaN,20230720,20231018,NaN,NaN,NaN,YES,NO,NATURAL,NaN,FIRST,90.0,",FD",NaN,NaN,NaN


In [3]:
#mm_data[mm_data['CURRENCY']=='USD']

In [81]:
mm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850 entries, 0 to 2849
Data columns (total 65 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UPLOAD_COMPANY     2850 non-null   object 
 1   @ID                2850 non-null   object 
 2   CUSTOMER_ID        2850 non-null   int64  
 3   ROLLOVER_MARKER    34 non-null     float64
 4   CURRENCY           2850 non-null   object 
 5   PRINCIPAL          2850 non-null   float64
 6   DEAL_DATE          2850 non-null   int64  
 7   VALUE_DATE         2850 non-null   int64  
 8   MATURITY_DATE      2850 non-null   int64  
 9   CATEGORY           2850 non-null   int64  
 10  INT_RATE_TYPE      2850 non-null   int64  
 11  INTEREST_BASIS     2850 non-null   object 
 12  INTEREST_RATE      2850 non-null   float64
 13  INTEREST_SPREAD_1  3 non-null      float64
 14  OUR_REMARKS        786 non-null    float64
 15  DRAWDOWN_ACCOUNT   2850 non-null   object 
 16  PRIN_LIQ_ACCT      2850 

# ACCOUNT

In [2]:
def ACCOUNT_TITLE(input_column):
    max_length = 35  # Maximum allowed length for the column
    cleaned_values = []
    for value in input_column:
        # Remove leading and trailing whitespaces
        value = str(value).strip()
        # Remove special characters except alphanumeric and spaces
        value = re.sub(r'[^a-zA-Z\s]', '', value)
        # Truncate the value if it exceeds the maximum length
        value = value[:max_length]
        cleaned_values.append(value)
    return cleaned_values
 
#Function to repeat values in Column2
def repeat_value(value, count):
    values = str(value).split('::')
    if len(values) < count:
        return '::'.join(values + [values[-1]] * (count - len(values)))
    else:
        return '::'.join(values[:count])
# Function to process individual dates
def process_date(date_str):
    if date_str == '!!':
        return pd.NaT
    date_obj = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
    if date_obj.year > 2040:
        return 20401230
    return int(date_obj.strftime('%Y%m%d')) if not pd.isnull(date_obj) else pd.NaT


def account_validation():
    # Clean the 'column_name' using the clean_column function
    data = pd.read_csv("R14 _DR3.ACCT.CUSTOMER.NEW.20230721.txt",sep='|',encoding='latin-1',dtype="object")
    print("Uncleaned data shape is ",data.shape)
    duplicate = pd.read_excel("Duplicate Customers.xlsx",dtype='object')
    #Create a mapping dictionary from the mapping_df
    mapping_dict = dict(zip(duplicate['DUPLICATES'], duplicate['CORRECT']))
    #Update the CustomerID column in customer_df using the mapping dictionary
    data['CUSTOMER'] = data['CUSTOMER'].map(mapping_dict).fillna(data['CUSTOMER']).astype(int)
    data['ACCOUNT_TITLE_1'] = ACCOUNT_TITLE(data['ACCOUNT_TITLE_1'])
    data['ACCOUNT_TITLE_2'] = ACCOUNT_TITLE(data['ACCOUNT_TITLE_2'])
    data['ALT_ACCT_ID'] = data['ALT_ACCT_ID'] + "::" + data['@ID']
    data['JOINT_NOTES'] = data.apply(lambda row: repeat_value(row['JOINT_NOTES'], len(str(row['JOINT_HOLDER']).split('::'))), axis=1)
    # Split dates separated by '!!' and apply processing function
    data['AML_ID_EDATE'] = data['AML_ID_EDATE'].fillna('')  # Fill NaN with empty string
    data['AML_ID_EDATE'] = data['AML_ID_EDATE'].str.split('!!').apply(lambda dates: '::'.join(map(str, [process_date(date) for date in dates])))
    print("Cleaned data shape is ",data.shape)
    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'cleaned_account_data.txt')
    data.to_csv(file_path,sep='|', index=False)
    return data

In [3]:
account_data = account_validation()

Uncleaned data shape is  (408437, 95)
Cleaned data shape is  (408437, 95)


In [6]:
account_data[account_data['CUSTOMER']==309044]

,UPLOAD_COMPANY,@ID,CUSTOMER,CATEGORY,ACCOUNT_TITLE_1,ACCOUNT_TITLE_2,SHORT_TITLE,MNEMONIC,CURRENCY,CURRENCY_MARKET,ACCOUNT_OFFICER,OTHER_OFFICER,POSTING_RESTRICT,RECONCILE_ACCT,INTEREST_LIQU_ACCT,INTEREST_COMP_ACCT,INT_NO_BOOKING,OPENING_DATE,JOINT_HOLDER,RELATION_CODE,JOINT_NOTES,PENDING_ID,STOCK_CONTROL_TYPE,LIQUIDATION_MODE,OVERDUE_STATUS,ER_BALANCE,ER_VALUE_DATE,SB_GROUP_ID,CREDIT_CHECK,TAX_SUSPEND,TAX_AT_SETTLE,EP_BALANCE,CONSOLIDATE_ENT,MAX_SUB_ACCOUNT,MASTER_ACCOUNT,HVT_FLAG,MANDATE_APPL,S_NAME,S_CLASS_OF_SIG,S_ID_TYPE,S_ID_NO,S_TELE,S_JOB_POS,M_NAME_BEN,M_EXP_LEVEL_DEP,M_EXP_LEVEL_FRE,M_EXP_ANN_INCO,M_TYPE_BUS,M_NAME_ASS_BUS,M_BUS_ADD,SOURCE_OF_FUNDS,PURPOSE_OF_AC,AC_SPEC_INST,IS_SIG_CUS,SMS_SUBSCRIBE,APPL_VERSION,MIG_OD_DATE,PREVIOUS_PROV,SMS_SUB_DATE,PREV_YEAR,INTRO_BY,MEMO_MESSAGE,AC_FUTURE_ID,EMAIL_SUBSCRIBE,END_DATE_MEMO,START_DATE_MEMO,C_SURNAME,C_OTHER_NAMES,C_DOB_ACCT,C_NATIONALITY,C_SEX,C_SCHOOL,C_RELATIONSHIP,TIN_NO,AML_GENDER,AML_NAME,AML_DOB,AML_POB,AML_MO_NAME,AML_NATIONALITY,AML_RES_ADD,AML_ID_TYPE,AML_ID_NO,AML_ID_COUNTRY,AML_ID_AUTH,AML_ID_SDATE,AML_ID_EDATE,AML_PHONE_NO,AML_EMP_NAME,SAL_IND,SALARY_COMP,BULK_SOURCE_ACC,DETAILS_CHECK,COMPLETION_DATE,ALT_ACCT_ID
70285,GH0010011,0122552525017,309044,6201,ADJEI EMMANUEL ANTWI,nan,ADJEI EMMANUEL ANTWI,NaN,GHS,1,107112,107108,NaN,NaN,NaN,NaN,NaN,20210715,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,NaN,6000,NaN,NaN,NaN,SERVICE PERSONAL,SAVINGS,SOLE TO SIGN,NaN,YES,NaN,NaN,NaN,20210715,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309618,GH0010011,0122309044019,309044,6201,ADJEI EMMANUEL ANTWI,nan,ADJEI EMMANUEL ANTWI,NaN,GHS,1,107112,107108,NaN,NaN,NaN,NaN,NaN,20160412,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,1200,NaN,NaN,NaN,SALARY,SAVINGS,SOLE TO SIGN,NaN,YES,NaN,NaN,NaN,20160412,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
nostros = pd.read_csv("R14 _DR3.CUST.NOST.20230721.txt",sep='|',dtype='object')
nostros.shape

(492, 307)

In [13]:
nostros.head()

,UPLOAD_COMPANY,@ID,MNEMONIC,SHORT_NAME,NAME_1,NAME_2,STREET,TOWN_COUNTRY,SECTOR,ACCOUNT_OFFICER,OTHER_OFFICER,INDUSTRY,TARGET,NATIONALITY,CUSTOMER_STATUS,RESIDENCE,CONTACT_DATE,INTRODUCER,TEXT,LEGAL_ID,REVIEW_FREQUENCY,BIRTH_INCORP_DATE,POSTING_RESTRICT,POST_CODE,COUNTRY,COMPANY_BOOK,CUSTOMER_RATING,ADDRESS,TITLE,GIVEN_NAMES,FAMILY_NAME,GENDER,DATE_OF_BIRTH,MARITAL_STATUS,NO_OF_DEPENDENTS,PHONE_1,SMS_1,EMAIL_1,EMPLOYMENT_STATUS,OCCUPATION,EMPLOYERS_NAME,EMPLOYERS_ADD,EMPLOYERS_BUSS,EMPLOYMENT_START,CUSTOMER_CURRENCY,SALARY,ANNUAL_BONUS,SALARY_DATE_FREQ,NET_MONTHLY_IN,NET_MONTHLY_OUT,RESIDENCE_STATUS,RESIDENCE_TYPE,RESIDENCE_SINCE,RESIDENCE_VALUE,FAX_1,PREVIOUS_NAME,CHANGE_DATE,CHANGE_REASON,CUSTOMER_SINCE,CUSTOMER_TYPE,DATE_LAST_VERIFIED,REL_DELIV_OPT,ROLE,ROLE_MORE_INFO,ROLE_NOTES,LEGAL_DOC_NAME,LEGAL_ISS_AUTH,LEGAL_ISS_DATE,LEGAL_EXP_DATE,OFF_PHONE,LEGAL_HOLDER_NAME,JOB_TITLE,PASTIMES,FURTHER_DETAILS,DOMICILE,OTHER_NATIONALITY,TAX_ID,RISK_LEVEL,RISK_TOLERANCE,LAST_KYC_REVIEW_DATE,AUTO_NEXT_KYC_REVIEW_DATE,MANUAL_NEXT_SUIT_REVIEW_DATE,LAST_SUIT_REVIEW_DATE,AUTO_NEXT_SUIT_REVIEW_DATE,KYC_RELATIONSHIP,MANDATE_APPL,MANDATE_REG,MANDATE_RECORD,SECURE_MESSAGE,AML_CHECK,AML_RESULT,LAST_AML_RESULT_DATE,KYC_COMPLETE,INTERNET_BANKING_SERVICE,MOBILE_BANKING_SERVICE,ALLOW_BULK_PROCESS,MERGED_TO,MERGED_STATUS,PERSON_ENTITY_ID,SEGMENT,CU_EFF_DATE,MAIDEN_NAME,MAJ_LAND_MARK,WEB_ADDRESS,FIRST_NAME,EMPL_FON,EMPL_EMAIL,SOCIAL_SEC_NO,SURNAME_NON_CUS,FIRST_NAME_NC,MIDDLE_NAME,DTE_OF_BIRTH_NC,GENDER_NC,RELATIONSHIP_NC,RESIDENT_ADD_NC,POSTAL_ADD_NC,MOBILE_NO_NC,ATM,PLACE_OF_INCORP,PARENT_COMP,PAR_COMP_INCORP,WEBSITE,VAT_REG_NO,CP_SURNAME,CP_FIRST_NAME,CP_DATEOFBIRTH,CP_GENDER,CP_OCCUPATION,CP_JOB_TITLE,CP_NATIONALITY,CP_RESID_ADD,CP_POST_ADDR,CP_MOB_NO,CP_EMAIL_ADD,PROOF_OF_ADD,ANNUAL_TURNOVER,SOURCE_OF_FUNDS,GSE_LISTING,GSE_NO,D_SURNAME,D_FIRST_NAME,D_GENDER,D_DATE_OF_BIRTH,D_OCCUPATION,D_NATIONALITY,D_RESI_PERMIT,D_ID_TYPE,D_ID_NUMBER,D_RESID_ADD,D_POST_ADDR,D_MOB_NO,D_EMAIL,D_DIR_STATUS,S_ENTITY_NAME,S_CONT_INCOR,S_BUS_ADDR,S_POST_ADDR,S_TEL,S_STATUS,S_BENEFI,S_REGI_CERT,S_EMAIL,C_TRANS_ALERT,C_STATE_PREF,C_STATE_FREQ,O_NAME_OF_BANK,O_BRANCH,O_ACCT_NAME,OTHER_ACCT,OTHER_ACCT_NO,NXT_KIN_SURNAME,NXT_KIN_FNAME,NXT_KIN_MNAME,NXT_KIN_DOB,NXT_KIN_GENDER,NXT_KIN_FON,NXT_KIN_MOB,COMP_REG_NO,PLACE_OF_BIRTH,REGION,CUS_PUR_ACCT,NUM_CURR,CUS_CHQ_REQ,CUS_CHQ_CONF,CUS_TRANS_ALERT,CUS_STMT_FRQ,CUS_CHQ_THRES,MOBILE_NUMBER,CUS_ACCT_BRANCH,CUS_ACCT_NAME,CUS_ACCT_NO,CUS_ACCT_TYPE,OTH_BNK_NAME,OTH_BNK_BRCH,OTH_ACC_NAME,OTH_ACC_NUM,OTH_ACC_TYPE,CUS_BUS_REG_NO,RESI_PER_NO,RESI_PER_DATE,CUS_PLA_ISSUE,CUS_TAX_ID,CUS_SSNIT_NO,CUS_M_MAIDEN_NA,CUS_PREV_NAME,CP_IDNO,APPL_POL_EXP,AUTH_POL_EXPO,CUS_DIR,IF_DIR_CUST,IND_SH_SN,IND_SH_FN,IND_SH_PA,IND_SH_PN,IND_SH_ST,CUS_ACCT_OTH_BK,APPL_VERSION,HSE_NO,CONT_PER,CONT_TEL,BUS_COM_DATE,BUS_COM_NUM,BUS_REG_DATE,CONTACT_NAME,CONT_PERSON,CONT_TELE,CONT_NAME,EZWICH_NO,RESIDENTIAL_ADD,POSTAL_ADD,CORR_NAME,ENTITY_TYPE,CUST_DUMMY,FOR_STATE,FOR_CITY,FOR_PHONE_HOUSE,FOR_MOB_NO,FOR_ADD,FOR_MAIL_ADD,FOR_TIN,FOR_SSN,FOR_GREEN_CARD,FOR_POA,FOR_INST,FOR_ZIP,FOR_CITIZEN,FOR_PERM_RESD,FOR_BIRTH_PLACE,FOR_COR_M_ADD,FOR_POA_SIGN,FOR_BORN,FOR_INST_FT,FOR_SUB_PRES,FOR_ENTITY,FOR_ENT_TYPE,FOR_ENT_OWNER,PO_REASON,TIN_NO,AMLCUS_GENDER,AMLCUS_NAME,AMLCUS_DOB,AMLCUS_PHONE_NO,AMLCUS_NATION,AMLCUS_RES_ADD,AMLCUS_OCC,AMLCUS_ROLE,AMLCUS_ID_TYPE,AMLCUS_ID_NO,AMLCUS_ID_CO,AMLCUS_ID_AUTH,AMLCUS_ID_SDATE,AMLCUS_ID_EDATE,CUST_SHAREHOLD,SHARE_NATIONAL,CUST_DIRECTOR,CUST_DIR_NA,CUST_SIGNATOR,CUST_SIG_NA,CUST_SIG_KCP,CUST_UBO,CUST_UBO_NA,CUST_EVT_INFO,CUST_ANNIVERSAR,NATURE_OF_BUSIN,NATURE_BUS_CERT,NATURE_EXPIRY,COMP_AC_TAG1,BUSINESS_ENTITY,RESE_PER_DATE,KYC_DEF_STATUS,KYC_DEF_REG_STA,CUS_CHAL_STATUS,CUS_REL_STATUS,CUS_REL_BANK,CUS_RES_STATUS,CUS_DIGI_ADD,DEP_CLASS_TYPE,PCUS_CARD_PURP,PCUS_DEP_FREQ,PCUS_DISABILITY,PCUS_EXP_LOAD,PCUS_F_SOURCE,PCUS_P_OF_BIRTH,PCUS_PRF_SOURCE,DETAILS_CHECK,REFER_INFLOW,GHA_CARD
0,GH0010001,999999,UM00999999,UNIVERSAL MERCHANT BANK LIMIT

In [14]:
nostros.DATE_OF_BIRTH.unique()

array([nan, '19830702', '19781018', '19780808', '19860525', '19560930',
       '19920508', '20210323', '19800402', '20180501', '19880420',
       '19770314', '19720501', '20210226', '19891007', '19720324',
       '19880922', '19941019', '19860319', '19850104', '19970101',
       '19820406', '19911210', '19930727', '19911027', '19840715',
       '19920124', '19900204', '19980512', '19821022', '19900213',
       '19811008', '19730519', '19880618', '19881002', '19741214',
       '19800423', '19900315', '19831001', '19810809', '19660805',
       '19991216', '19890812', '19930511', '19820707', '19950504',
       '19910325', '19920610', '19830724', '19591206', '19950618',
       '19870122', '19880426', '19980507', '19710613', '19930921',
       '19921130', '19821123', '19850706', '19840907', '19920203',
       '19830727', '19831210', '19920926', '19930406', '19850128',
       '19840422', '19980709', '19840703', '19940701', '19870831',
       '19840115', '19931006', '19840517', '20160425', '1

In [24]:
update_customer = nostros[~nostros["DATE_OF_BIRTH"].isna()]

In [25]:
update_customer.shape

(88, 307)

## LETTERS OF CREDIT

In [57]:
import pandas as pd
def LETTERS OF CREDIT(filename):
    df = pd.read_csv(filename,delimiter = "|", dtype = "object",low_memory = False)
    # Replace commas in the 'LC_USD_AMT' column
    df['LC_USD_AMT'] = df['LC_USD_AMT'].str.replace(',', '')

    # Replace commas in the 'LC_USD_AMT_9' column
    df['LC_USD_AMT.1'] = df['LC_USD_AMT.1'].str.replace(',', '')
    
    Closed_LC = pd.read_excel(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\REVISED FILES\DATA NOT TO BE MIGRATED IN R22.xlsx", dtype = "object")
    Exclusions = df[~df["@ID"].isin(Closed_LC["Transaction Ref"])]

    return df

## GUARANTEES

In [ ]:
import pandas as pd
def GUARANTEES(filename):
    df = pd.read_csv(filename,delimiter = "|", dtype = "object",low_memory = False)
    # Replace commas in the 'LC_USD_AMT' column
    df['LC_USD_AMT'] = df['LC_USD_AMT'].str.replace(',', '')
    return df

## DRAWINGS

In [ ]:
import pandas as pd
def DRAWINGS(filename):
    drawings = pd.read_csv(filename,delimiter = "|", dtype = "object",low_memory = False)
    # Replace commas in the 'LC_USD_AMT' column
    drawings['LC_USD_AMT'] = drawings['LC_USD_AMT'].str.replace(',', '')
    
    # Create a new column with the new LC number
    drawings["lookup"] = drawings["@ID"].str[:12]
    
    # Filter for only records that exist in LC dataset
    lc_file = lc_file.filter(["@ID"]).rename(columns = {"@ID" :"lookup"})
    
    # Create a new dataset with only the records they share in common
    df = pd.merge(drawings,lc_file, on = "lookup",how = "inner")
    
    # Drop the new columns that was created for looking up the contracts
    df = df.drop(columns = "lookup")
    
    return df

In [104]:
file = pd.read_csv(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\ORIGINAL FILES\FINANCIAL\R14 _DR3.UMB.DRAWINGS.20230721.txt",delimiter = "|",dtype = "object")

In [105]:
file

,@ID,LC_NUMBER,DRAWING_TYPE,DRAW_CURRENCY,DOCUMENT_AMOUNT,REDUCE_LIMIT_BAL,PARENT_DRAWING,CHILD_DRAWS,MATURITY_REVIEW,NUMBER_OF_DAYS,...,SHIPMENT_DATE,VESSEL_NAME,PORT_OF_LOADING,COUNTRY_1,PORT_OF_DISCHAR,COUNTRY,TENOR_STRTDATE,TENOR_PERIOD,LC_NUMBER_REF,BEN_NAME_DR
0,TF1600102050161,TF1600102050161,SP,USD,162.93,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF160018860025,TF160018860025,SP,GBP,9300.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TF161440179377,TF161440179377,SP,EUR,3728.74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TF1600102050109,TF1600102050109,SP,USD,5818.56,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TF1600102050111,TF1600102050111,SP,USD,2207.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2284,TF232024541001,TF232024541001,SP,USD,9000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2285,TF231284417501,TF231284417501,SP,USD,150000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2286,TF231574468101,TF231574468101,SP,USD,164000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2287,TF231594472501,TF231594472501,SP,USD,42250.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
file["lookup"] = file["@ID"].str[:12]

In [107]:
file

,@ID,LC_NUMBER,DRAWING_TYPE,DRAW_CURRENCY,DOCUMENT_AMOUNT,REDUCE_LIMIT_BAL,PARENT_DRAWING,CHILD_DRAWS,MATURITY_REVIEW,NUMBER_OF_DAYS,...,VESSEL_NAME,PORT_OF_LOADING,COUNTRY_1,PORT_OF_DISCHAR,COUNTRY,TENOR_STRTDATE,TENOR_PERIOD,LC_NUMBER_REF,BEN_NAME_DR,lookup
0,TF1600102050161,TF1600102050161,SP,USD,162.93,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF1600102050
1,TF160018860025,TF160018860025,SP,GBP,9300.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF1600188600
2,TF161440179377,TF161440179377,SP,EUR,3728.74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF1614401793
3,TF1600102050109,TF1600102050109,SP,USD,5818.56,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF1600102050
4,TF1600102050111,TF1600102050111,SP,USD,2207.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF1600102050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2284,TF232024541001,TF232024541001,SP,USD,9000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF2320245410
2285,TF231284417501,TF231284417501,SP,USD,150000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF2312844175
2286,TF231574468101,TF231574468101,SP,USD,164000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF2315744681
2287,TF231594472501,TF231594472501,SP,USD,42250.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TF2315944725


In [68]:
# Closed_LC = pd.read_excel(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\REVISED FILES\DATA NOT TO BE MIGRATED IN R22.xlsx", dtype = "object")
# Closed_LC = Closed_LC.rename(columns = {"Transaction Ref":"lookup" })

In [93]:
# Closed_LC

In [108]:
lc_file = pd.read_csv(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\ORIGINAL FILES\FINANCIAL\R14 _DR3.UMB.LC.202307021.txt",delimiter = "|",dtype = "object").filter(["@ID"]).rename(columns = {"@ID" :"lookup"})


In [109]:
lc_file

,lookup
0,TF1600143562
1,TF1733211504
2,TF2316644830
3,TF2030127103
4,TF2106829665
...,...
1009,TF2220239820
1010,TF2227140902
1011,TF2214538834
1012,TF2208237674


In [110]:
new_drawings = pd.merge(file,lc_file, on = "lookup",how = "inner")

In [113]:
new_drawings = new_drawings.drop(columns = "lookup")

In [114]:
new_drawings

,@ID,LC_NUMBER,DRAWING_TYPE,DRAW_CURRENCY,DOCUMENT_AMOUNT,REDUCE_LIMIT_BAL,PARENT_DRAWING,CHILD_DRAWS,MATURITY_REVIEW,NUMBER_OF_DAYS,...,SHIPMENT_DATE,VESSEL_NAME,PORT_OF_LOADING,COUNTRY_1,PORT_OF_DISCHAR,COUNTRY,TENOR_STRTDATE,TENOR_PERIOD,LC_NUMBER_REF,BEN_NAME_DR
0,TF161650213701,TF161650213701,SP,USD,10350.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF160014361502,TF160014361502,SP,USD,2000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TF160014361514,TF160014361514,SP,USD,45000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TF160014361513,TF160014361513,SP,USD,2000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TF160014361504,TF160014361504,SP,USD,2000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,TF210082861501,TF210082861501,SP,USD,3675.49,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,TF220743749201,TF220743749201,SP,USD,9726.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
667,TF231664482401,TF231664482401,SP,USD,50000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,TF223564228501,TF223564228501,SP,USD,65000.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
LC = read_and_replace_commas(file)

In [61]:
LC["LC_USD_AMT.1"]

0           178710
1        832043.00
2       1315405.00
3        640095.75
4        720193.16
           ...    
1009    2520598.87
1010    4274400.00
1011     454860.00
1012     317200.00
1013     108484.00
Name: LC_USD_AMT.1, Length: 1014, dtype: object

## Account Balance

In [ ]:
def account_balance():
    account_balance = pd.read_csv("R14_DR3.ONLINE.CLEARED.CUS.txt",sep="|",encoding="latin-1",dtype="object")
    account_balance.shape
    account_balance['CREDIT.AMOUNT'] = account_balance['CREDIT.AMOUNT'].astype(float)
    positive_bal = account_balance[account_balance['CREDIT.AMOUNT'] > 0 ]
    negative_bal = account_balance[account_balance['CREDIT.AMOUNT'] < 0 ]
    account_balance_edit = pd.concat([positive_bal,negative_bal],ignore_index=True)
    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'cleaned_account_balance.txt')
    account_balance_edit.to_csv(file_path,sep='|', index=False)
    return account_balance_edit

 

clean = account_balance()

## GL BALANCE

In [21]:
def gl():
    gl_data = pd.read_csv(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\ORIGINAL FILES\FINANCIAL\R14_DR3.ONLINE.CLEARED.GL.txt",sep="|",dtype="object", encoding='latin-1') 
    gl_data = gl_data[gl_data['CREDIT.ACCT.NO'].str.len() < 19]
    gl_data = gl_data[gl_data['DEBIT.ACCT.NO'].str.len() < 19]
    gl_data["CUSTOMER_ID"] = gl_data['DEBIT.ACCT.NO'].str[3:8]
    customer_id_drop = [10001,14000,15005, 15006, 15007,15009,15010,12030,15008]
    # Delete rows with customer_id_drop values in the numbers_to_delete list
    gl_data = gl_data[~gl_data["CUSTOMER_ID"].astype(int).isin(customer_id_drop)]
    gl_data = gl_data.drop("CUSTOMER_ID",axis=1)
    gl_data = pd.read_csv(r"C:\Users\KATAKOF\Downloads\T24 DRESS REHEARSAL 3\ORIGINAL FILES\FINANCIAL\R14_DR3.ONLINE.CLEARED.GL.txt",sep="|",dtype="object", encoding='latin-1') 
    branch_code = pd.read_excel(r"C:\Users\KATAKOF\Downloads\Brnch Codes.xlsx",dtype="object")
    gl_data = gl_data[gl_data['CREDIT.ACCT.NO'].str.len() < 19]
    gl_data = gl_data[gl_data['DEBIT.ACCT.NO'].str.len() < 19]
    ### get the customer id
    gl_data["CUSTOMER_ID"] = gl_data['DEBIT.ACCT.NO'].str[3:8]
    customer_id_drop = [10001,14000,18048]
#                         15005, 15006, 15007,15009,15010,12030,15008]
    # Delete rows with customer_id_drop values in the numbers_to_delete list
    gl_data = gl_data[~gl_data["CUSTOMER_ID"].astype(int).isin(customer_id_drop)]
    #gl_data = gl_data.drop("CUSTOMER_ID",axis=1)
    gl_data.shape
    gl_data_new = gl_data[gl_data['CUSTOMER_ID']=='14998']
    print(gl_data_new.shape)
    gl_data_old = gl_data[gl_data['CUSTOMER_ID']!='14998']
    print(gl_data_old.shape)
    gl_data_old["old_code"] = gl_data_old['CREDIT.ACCT.NO'].str[-2:]
    #gl_data_old.head()
    ### CREDIT.ACCT.NO
    old_gl = gl_data_old.merge(branch_code, on="old_code",how='left')
    old_gl.shape
    old_gl["CREDIT.ACCT.NO"] = old_gl["CREDIT.ACCT.NO"].str[:-2] + old_gl['new_code']
    cleaned_old_cre = old_gl.drop(['Old','New','Mneumonics','new_code','old_code'],axis=1)
    gl_data_1 = pd.concat([gl_data_new,cleaned_old_cre])
    #print(gl_data_1.head())
    ### Debit Account
    gl_data_1["old_code"] = gl_data_1['DEBIT.ACCT.NO'].str[-2:]
    gl_data_1 = gl_data_1.merge(branch_code, on="old_code",how='left')
    gl_data_1["DEBIT.ACCT.NO"] = gl_data_1['DEBIT.ACCT.NO'].str[:-2] + gl_data_1['new_code']
    cleaned_old_debit = gl_data_1.drop(['Old','New','Mneumonics','new_code','old_code'],axis=1)

    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'GL_data.txt')
    cleaned_old_debit.to_csv(file_path,sep='|', index=False)
    return {"gl_data_1":gl_data_1, "cleaned_old_debit":cleaned_old_debit}

 

gl_data_1,gl_data_2 = gl()["gl_data_1"], gl()["cleaned_old_debit"]

(2721, 10)
(722, 10)
(2721, 10)
(722, 10)


## ADI

In [3]:
import pandas as pd
import numpy as np
def adi():
    adi_data = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\GOLIVE DATA\ORIGINAL FILES\FINANCE\R14 _GOLIVE.ACI.DATA.20230803.txt",sep="|",encoding="latin-1",dtype="object")
    print('Loaded data shape',adi_data.shape)
    adi_data["date"] = adi_data["@ID"].str[-8:]
    adi_data["account"] = adi_data["@ID"].str[:-9]

    adi_data_dupl = adi_data.sort_values(by=['account', 'date'], ascending=[True, True])
    single_data = adi_data_dupl[~adi_data_dupl.duplicated(subset=['account'], keep=False)]
    single_data = single_data.drop(["account","date"],axis=1)

 

    duplicates_data = adi_data_dupl[adi_data_dupl.duplicated(subset=['account'], keep=False)]
    duplicates_data = duplicates_data.sort_values(by=['account', 'date'], ascending=[True, True])
    # Identify the maximum age for each duplicated name
    duplicates_data['date'] = duplicates_data['date']#.astype(int)
    max_age_per_name = duplicates_data.groupby('account')['date'].transform('max')
    #max_age_per_name = pd.DataFrame(max_age_per_name,columns=["date"])
    # Update the 'Age' column to 100 for rows where the age is equal to the maximum age per name
    duplicates_data.loc[(duplicates_data['date'] == max_age_per_name), 'date'] = "20230807"
    duplicates_data["IDe"] = duplicates_data["account"]+"-"+duplicates_data["date"]
    duplicates_data = duplicates_data.drop(["@ID","account","date"],axis=1)
    duplicates_data.rename(columns = {"IDe":"@ID"},inplace=True)
    columns = ['UPLOAD_COMPANY', '@ID','CHARGE_KEY', 'INTEREST_DAY_BASIS', 'TAX_KEY',
           'DR_BALANCE_TYPE', 'DR_CALCUL_TYPE', 'DR_BASIC_RATE', 'DR_INT_RATE',
           'DR_MARGIN_OPER', 'DR_MARGIN_RATE', 'DR_LIMIT_AMT', 'DR2_BALANCE_TYPE',
           'DR2_CALCUL_TYPE', 'DR2_BASIC_RATE', 'DR2_INT_RATE', 'DR2_MARGIN_OPER',
           'DR2_MARGIN_RATE', 'DR2_LIMIT_AMT', 'APR_REQUIRED', 'MAX_LEGAL_RATE',
           'LINK_TO_GROUP', 'MAX_DEBIT_CHG_RATE', 'DR_MIN_RATE', 'DR2_MIN_RATE',
           'DR_MAX_RATE', 'DR2_MAX_RATE', 'DR_MIN_VALUE', 'DR_MIN_WAIVE',
           'DR2_MIN_VALUE', 'DR2_MIN_WAIVE', 'COMPOUND_TYPE',
           'COMPOUND_YIELD_METHOD']
    duplicates_data = duplicates_data[columns]
    ### concat the single and duplicates
    final_adi_data = pd.concat([single_data,duplicates_data])
    print('Final Data shape',final_adi_data.shape)

    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'ACI_data.txt')
    final_adi_data.to_csv(file_path,sep='|', index=False)
    return final_adi_data

 

adi_data = adi()

Loaded data shape (335, 13)


KeyError: "['CHARGE_KEY', 'DR_BALANCE_TYPE', 'DR_CALCUL_TYPE', 'DR_BASIC_RATE', 'DR_INT_RATE', 'DR_MARGIN_OPER', 'DR_MARGIN_RATE', 'DR_LIMIT_AMT', 'DR2_BALANCE_TYPE', 'DR2_CALCUL_TYPE', 'DR2_BASIC_RATE', 'DR2_INT_RATE', 'DR2_MARGIN_OPER', 'DR2_MARGIN_RATE', 'DR2_LIMIT_AMT', 'APR_REQUIRED', 'MAX_LEGAL_RATE', 'LINK_TO_GROUP', 'MAX_DEBIT_CHG_RATE', 'DR_MIN_RATE', 'DR2_MIN_RATE', 'DR_MAX_RATE', 'DR2_MAX_RATE', 'DR_MIN_VALUE', 'DR_MIN_WAIVE', 'DR2_MIN_VALUE', 'DR2_MIN_WAIVE', 'COMPOUND_TYPE', 'COMPOUND_YIELD_METHOD'] not in index"

## COLLATERAL

In [4]:
def colateral():    
    col_data = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\GOLIVE DATA\ORIGINAL FILES\FINANCE\R14 _GOLIVE.COL.20230803.txt",sep='|',encoding='latin-1',dtype="object")
    col_data["COMPANY"].replace("NRD","OSU",inplace=True)
    # Assuming you have a DataFrame named col_data
    col_data["EXECUTION_VALUE"] = np.where(col_data["EXECUTION_VALUE"] == ".", "", col_data["EXECUTION_VALUE"])
    col_data['APPLICATION_ID'] = col_data['APPLICATION_ID'].str.replace(r'\*NO_RECORD$', '', regex=True)
    # Create a folder (if it doesn't exist)
    folder_name = 'cleaned_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

 

    # Export DataFrame data to a CSV file inside the folder
    file_path = os.path.join(folder_name, 'colateral_data.txt')
    col_data.to_csv(file_path,sep='|', index=False)

    return col_data

 

colateral_data = colateral()

## COLLATERAL RIGHT

In [ ]:
def colateral_right():
    col_rigth_data = pd.read_csv("R14 _DR3.UMB.COLRIG.20230721.txt",sep="|",dtype="object",encoding="latin-1")
    col_rigth_data["COMPANY"].replace("NRD","OSU",inplace=True)